# 📦 1. Import necessary libraries

In [ ]:
import requests
import json
from datetime import timedelta
import pandas as pd
from ApiKey import api_key
import os

# ------------------- CONFIG -------------------

# 🔐 2. Configure API key, headers, players name and url

In [ ]:
# We create the headers that PUBG requires to authenticate and specify the format
HEADERS = {
    "Authorization": f"Bearer {api_key}",
    "Accept": "application/vnd.api+json"
}

# Name of player to search
player_name  = 'arbigegegegegege'
#player_name  = 'KaDiz-'
#player_name  = 'XioYuanLinPay'

# URL base to the PUBG API
base_url = "https://api.pubg.com/shards/steam"

# ------------------- EXTRACT -------------------

# 🎮 3. Find player ID

In [ ]:
def searchPlayer (player_name):
    # Endpoint to find player for name
    url = f"{base_url}/players?filter[playerNames]={player_name}"
    
    # Send the GET request to the API with authorization headers
    response = requests.get(url, headers=HEADERS)

    # Raise an exception if the request failed (e.g., 404 or 500)
    response.raise_for_status()    

    #Save the information and transform with Json library and return the player id
    data = response.json()
    return data["data"][0]["id"]
    
searchPlayer(player_name)

# 🎯 4. Obtain matches from the player

In [ ]:
def find_match_players(player_id):
    # Endpoint to obtain data of the player from id
    url = f"{base_url}/players/{player_id}"

    # Send the GET request to the API with authorization headers
    response = requests.get(url, headers=HEADERS)
    
    # Raise an exception if the request failed (e.g., 404 or 500)
    response.raise_for_status()  

    # Returns all of matches from the player
    return response.json()["data"]["relationships"]["matches"]["data"]

find_match_players(searchPlayer(player_name))


# 📊 5. Obtain match details

In [ ]:
def extract_match_details(match_id):
    # Construct the API endpoint URL using the match ID
    url = f"{base_url}/matches/{match_id}"
    
    # Send the GET request to the API with authorization headers
    response = requests.get(url, headers=HEADERS)
    
    # Raise an exception if the request failed (e.g., 404 or 500)
    response.raise_for_status()
    
    # Return the JSON response containing match details
    return response.json()

extract_match_details(find_match_players(searchPlayer(player_name))[0]['id'])


# 📊 6. Obtain telemetry details

In [ ]:
def download_telemetry(match_id):
    url = f"{base_url}/matches/{match_id}"
    response = requests.get(url, headers=HEADERS)
    response.raise_for_status()
    match_data = response.json()

    # Obtener asset ID de telemetría
    asset_id = match_data['data']['relationships']['assets']['data'][0]['id']

    # Buscar el asset en el array 'included'
    asset = next((item for item in match_data['included'] if item['type'] == 'asset' and item['id'] == asset_id), None)
    if asset is None:
        print("❌ Telemetry asset not found.")
        return match_data

    telemetry_url = asset['attributes']['URL']
    
    tele_response = requests.get(telemetry_url, headers={"Accept-Encoding": "gzip"})
    tele_response.raise_for_status()

    os.makedirs("telemetry", exist_ok=True)
    filepath = f"telemetry/{match_id}_telemetry.json"
    with open(filepath, "wb") as f:
        f.write(tele_response.content)

    print(f"✅ Telemetría guardada en {filepath}")

download_telemetry(find_match_players(searchPlayer(player_name))[0]['id'])

# ------------------- TRANSFORM -------------------

In [ ]:
def player_match_info(match_data):
    participants = [x for x in match_data['included'] if x['type'] == 'participant']
    rows = []
    for p in participants:
        stats = p['attributes']['stats']
        rows.append({
            "name": stats['name'],
            "playerId": stats["playerId"],
            "kills": stats['kills'],
            "longestKill": stats["longestKill"],
            "killPlace": stats["killPlace"],
            "assists": stats["assists"],
            "damageDealt": stats['damageDealt'],
            "winPlace": stats['winPlace'],
            "timeSurvived": str(pd.to_timedelta(stats['timeSurvived'], unit='s')).split()[-1],
            "heals": stats["heals"],
            "boosts": stats["boosts"],
            "deathType": stats["deathType"],
            "walkDistance": stats["walkDistance"],
            "rideDistance": stats["rideDistance"],
            "swimDistance": stats["swimDistance"],
            "vehicleDestroys": stats["vehicleDestroys"]
        })
    return pd.DataFrame(rows)

player_match_info(extract_match_details(find_match_players(searchPlayer(player_name))[0]['id']))

In [ ]:
def match_info(match_data):
    data = match_data['data']['attributes']
    rows = []
    rows.append({
            "gameMode":data['gameMode'],
            "mapName": data["mapName"],
            "createdAt": data['createdAt'],
            "isCustomMatch": data["isCustomMatch"],
            "matchType": data["matchType"],
            "seasonState": data["seasonState"]
    })
        
    return pd.DataFrame(rows)


match_info(extract_match_details(find_match_players(searchPlayer(player_name))[1]['id']))

In [ ]:
def rooster_info(match_data):
    rosters = match_data['data']['relationships']['rosters']['data']
    rows = []
    for r in rosters:
        rows.append({
            "id": r['id']
        })
        
    return pd.DataFrame(rows)


rooster_info(extract_match_details(find_match_players(searchPlayer(player_name))[1]['id']))

# 📊 5.2 Mostrar la media de los ultimos 10 matchs

In [ ]:
# Variables para acumular los resultados
total_kills = 0
total_damage = 0
total_time_survived = 0
total_ranking = 0
total_wins = 0
total_kill_place = 0
total_kill_streaks = 0
num_matches = 0  # Para contar el número de partidas procesadas

# Recorremos cada match_id para obtener información de la partida
for match_id  in match_ids:
    # Endpoint para obtener datos del match
    url = f"https://api.pubg.com/shards/steam/matches/{match_id}"
    
    # Hacemos la petición para obtener los datos de la partida
    response = requests.get(url, headers=headers)
    
    # Convertimos la respuesta a JSON
    match_data = response.json()
    
    # Accedemos a la información del jugador dentro de esa partida
    participants = match_data["included"]

    # Buscamos al jugador en esa partida (comparando player_id)
    for player in participants:
        if player["type"] == "participant" and "stats" in player["attributes"]:
            stats = player["attributes"]["stats"]
           
            # Verifica si el jugador es el que estás buscando
            if stats.get("playerId") == player_id:
                kills = stats.get("kills", 0)
                killPlace = stats.get("killPlace", 0)
                killStreaks = stats.get("killStreaks", 0)
                damage = stats.get("damageDealt", 0)
                duracion = timedelta(seconds=stats.get("timeSurvived", 0))
                ranking = stats.get("winPlace", 0)
                win = ranking == 1
                
                # Acumular los valores
                total_kills += kills
                total_damage += damage
                total_time_survived += stats.get("timeSurvived", 0)
                total_ranking += ranking
                total_wins += win
                total_kill_place += killPlace
                total_kill_streaks += killStreaks
                num_matches += 1

# Calcular las medias
if num_matches > 0:
    avg_kills = total_kills / num_matches
    avg_damage = total_damage / num_matches
    avg_time_survived = timedelta(seconds=(total_time_survived / num_matches))
    avg_ranking = total_ranking / num_matches
    avg_wins = total_wins / num_matches
    avg_kill_place = total_kill_place / num_matches
    avg_kill_streaks = total_kill_streaks / num_matches

    # Mostrar las medias
    print("Average Stats over the last 10 matches:")
    print(f"Average Kills: {avg_kills:.2f}")
    print(f"Average Damage: {avg_damage:.2f}")
    print(f"Average Time Survived: {avg_time_survived}")
    print(f"Average Ranking: {avg_ranking:.2f}")
    print(f"Average Wins: {avg_wins:.2f}")
    print(f"Average Kill Place: {avg_kill_place:.2f}")
    print(f"Average Kill Streaks: {avg_kill_streaks:.2f}")
else:
    print("No matches found.")

# 📊 5.5 Mostrar detalles del primer match obteniendo todos los jugadores y equipos

In [ ]:
# Recorremos cada match_id para obtener información de la partida
for match_id  in last_Match_id:
    # Endpoint para obtener datos del match
    url = f"https://api.pubg.com/shards/steam/matches/{match_id}"
    
    # Hacemos la petición para obtener los datos de la partida
    response = requests.get(url, headers=headers)
    
    # Convertimos la respuesta a JSON
    match_data = response.json()
    
    # Buscamos al jugador en esa partida (comparando player_id)
    roster_match = match_data["included"]
    all_participants = []  # ← Aquí guardaremos todos los participantes

    # Primero, recogemos todos los participantes
    for data in roster_match:
        if data["type"] == "participant":
            all_participants.append(data)

    # Usamos un set para llevar control de los jugadores ya procesados y evitar duplicados
    seen_players = set()

    # Suponiendo que tienes una lista de rosters
    for data in roster_match:  
        if data["type"] == "roster":
            team_id = data["id"]
            team_rank = data["attributes"]["stats"].get("rank", "Desconocido")
            team_Id = data["attributes"]["stats"].get("teamId", "Desconocido")
            team_win = data["attributes"]["won"] == "true"
            participants = data["relationships"]["participants"]["data"]

            print(f"Match ID: {match_id}")
            print(f"team id: {team_id}")
            print(f"team nº: {team_Id}")
            print(f"team rank: {team_rank}")
            print(f"Win?: {'Yes' if team_win else 'No'} \n")   

            #Buscar jugadores en el roster y sus estadísticas
            for player_info in participants:
                player_id = player_info["id"]  # ID del jugador en el roster
                
                # Evitamos imprimir jugadores repetidos
                if player_id in seen_players:
                    continue
                seen_players.add(player_id)  # Lo marcamos como "visto"

                # Buscar el participante con ese ID
                player = next((p for p in all_participants if p["id"] == player_id), None)

                if not player or "attributes" not in player or "stats" not in player["attributes"]:
                    continue

                stats = player["attributes"]["stats"]
                kills = stats.get("kills", 0)
                name_player = stats.get("name", "Desconocido")
                damage = stats.get("damageDealt", 0)
                ranking = stats.get("winPlace", 0)
                duracion = timedelta(seconds=stats.get("timeSurvived", 0))
                win = ranking == 1

                # Mostrar detalles del jugador
                print(f"Player ID: {player_id}")
                print(f"Player Name: {name_player}")
                print(f"Kills: {kills}")
                print(f"Damage: {damage}")
                print(f"Time survived: {duracion}")
                print(f"Ranking: {ranking} \n")

# Adaptando el codigo a Pandas

In [ ]:
print(json.dumps(dataOriginal, indent=4))

In [ ]:
# Suponiendo que tu variable es 'data' 
players = dataOriginal["data"]

# Normalizamos todo lo posible, separando las claves anidadas con "_"
df = pd.json_normalize(players, sep="_")

# Mostramos el DataFrame completo
df


In [ ]:
# Normalizar los matches dentro de relationships.matches.data
df_matches = pd.json_normalize(
    players,
    record_path=["relationships", "matches", "data"],
    sep="_"
)

# Mostramos el DataFrame completo
df_matches

In [ ]:
# URL para obtener las seasons
urlSeasonActual = "https://api.pubg.com/shards/steam/seasons"


# Hacemos la petición
responseActualSeason = requests.get(urlSeasonActual, headers=headers)

# Verificamos y extraemos el seasonId actual
if responseActualSeason.status_code == 200:
    seasons = responseActualSeason.json()
    
    # Buscamos la temporada actual
    season_id = None
    for s in seasons["data"]:
        if s["attributes"].get("isCurrentSeason"):
            season_id = s["id"]
            break

    if season_id is None:
        print("No se encontró la temporada actual.")
    else:
        # Armamos el endpoint con el season actual
        endpointSeason = f"https://api.pubg.com/shards/steam/players/{player_id}/seasons/{season_id}/ranked"

        # Hacemos la petición
        response = requests.get(endpointSeason, headers=headers)

        if response.status_code == 200:
            dataSeason = response.json()
            print("Todo Correcto, arcivhos guardado en data")
        else:
            print("Error en ranked:", response.status_code)
else:
    print("Error al obtener seasons:", responseActualSeason.status_code)


In [ ]:
print(json.dumps(dataSeason, indent=4))